In [1]:
import requests
import pandas as pd
import cvxpy as cp
import numpy as np

In [2]:
print('cvxpy', cp.__version__)
print('numpy', np.__version__)
print('pandas', pd.__version__)

cvxpy 1.0.31
numpy 1.19.1
pandas 1.1.0


# Downloading of MOEX Russia Index  

Weights of assets

In [3]:
analytics_url = \
'http://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics/IMOEX.json?start={i}&analytics.columns={columns}'

columns = ['ticker', 'weight']

res = requests.get(analytics_url.format(i=0, columns=','.join(columns)))
res_json = res.json()

pagesize_index = res_json['analytics.cursor']['columns'].index('PAGESIZE')
pagesize = res_json['analytics.cursor']['data'][0][pagesize_index]

total_index = res_json['analytics.cursor']['columns'].index('TOTAL')
total = res_json['analytics.cursor']['data'][0][total_index]

In [4]:
analitics_rows = []
for i in range(0, total, pagesize):
    res = requests.get(analytics_url.format(i=i, columns=','.join(columns)))
    res_json = res.json()
    analitics_rows += res_json['analytics']['data']

In [5]:
analytics_data = pd.DataFrame(analitics_rows, columns=columns)
analytics_data.head()

,ticker,weight
0,AFKS,0.50
1,AFLT,0.29
2,ALRS,1.38
3,CBOM,0.28
4,CHMF,1.24


Current prices of assets

In [6]:
securities_metadata_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/metadata.json'

securities_url = \
'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json?securities={securities}'

res = requests.get(securities_metadata_url)
res_json = res.json()

lotsize_index = res_json['securities']['columns'].index('LOTSIZE')
lastprice_index = res_json['marketdata']['columns'].index('LAST')

In [7]:
lotsize_rows = []
lastprice_rows = []
for i in range(0, total, 10):
    securities = ','.join(analytics_data.ticker[i:i+10])
    res = requests.get(securities_url.format(securities=securities))
    res_json = res.json()
    for securities_row in res_json['securities']['data']:
        lotsize_rows.append(securities_row[lotsize_index])
    for marketdata_row in res_json['marketdata']['data']:
        lastprice_rows.append(marketdata_row[lastprice_index])

Index table

In [8]:
analytics_data['lotsize'] = lotsize_rows
analytics_data['lastprice'] = lastprice_rows
analytics_data['weight'] = analytics_data['weight'] * 0.01
analytics_data

,ticker,weight,lotsize,lastprice
0,AFKS,0.0050,100,20.097000
1,AFLT,0.0029,10,82.000000
2,ALRS,0.0138,10,69.920000
3,CBOM,0.0028,100,5.927000
4,CHMF,0.0124,1,938.800000
5,DSKY,0.0039,10,121.200000
6,FEES,0.0036,10000,0.201020
7,FIVE,0.0249,1,2842.000000
8,GAZP,0.1292,10,188.310000
9,GMKN,0.0716,1,20786.000000


# Portfolio generation

Input parameters:

* $T$ - total amount of investment (in rubles)
* $a$ - weights of assets in the index
* $p$ - current prices of assets
* $l$ - numbers of assets in a single lot
* $n$ - a number of assets in the index

Variables:

* $x$ - weights of assets in the portfolio
* $y$ - numbers of lots

Solve the optimization problem with constraints:

$$f(x) = \sum_i^n (x_i - a_i)^2 \to \min$$

$$\begin{cases}
\sum_i^n x_i \leq 1 \\
\forall i: x_i \geq 0 \\
\forall i: y_i = T x_i\frac{1}{p_i} \frac{1}{l_i}\\
\forall i: y_i \in \mathbb Z
\end{cases}$$

In [9]:
total = 100000
weight = analytics_data.weight.values
price = analytics_data.lastprice.values
lotsize = analytics_data.lotsize.values

x_weight = cp.Variable(analytics_data.shape[0], nonneg=True)
y_count = cp.Variable(analytics_data.shape[0], integer=True)

obj = cp.Minimize(cp.sum_squares(x_weight - weight))
constr1 = (y_count == total * x_weight / price / lotsize)
constr2 = (cp.sum(x_weight) <= 1)

problem = cp.Problem(obj, [constr1, constr2])
problem.solve(solver=cp.ECOS_BB)
problem.status

'optimal_inaccurate'

Portfolio table

In [10]:
portfolio = pd.DataFrame(analytics_data[['ticker']])
portfolio['weight'] = x_weight.value.round(4)
portfolio['count'] = y_count.value * analytics_data.lotsize
portfolio['total'] = y_count.value * analytics_data.lotsize * analytics_data.lastprice
portfolio

,ticker,weight,count,total
0,AFKS,0.0000,0.0,0.00
1,AFLT,0.0000,0.0,0.00
2,ALRS,0.0140,20.0,1398.40
3,CBOM,0.0000,0.0,0.00
4,CHMF,0.0094,1.0,938.80
5,DSKY,0.0000,0.0,0.00
6,FEES,0.0000,0.0,0.00
7,FIVE,0.0284,1.0,2842.00
8,GAZP,0.1318,70.0,13181.70
9,GMKN,0.0000,0.0,0.00


Total portfolio value

In [11]:
round(portfolio.total.sum(), 2)

89459.05